In [ ]:
import cv2
import numpy as np
import PIL
import matplotlib.pyplot as plt
import keras
from keras.applications import vgg16, inception_v3, resnet50, mobilenet
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.imagenet_utils import decode_predictions

%matplotlib inline

from PIL import Image

In [ ]:
#Load the VGG model
vgg_model = vgg16.VGG16(weights='imagenet')

#Load the ResNet50 model
resnet_model = resnet50.ResNet50(weights='imagenet')

#Load the MobileNet model
mobilenet_model = mobilenet.MobileNet(weights='imagenet')

def predict(image, model=vgg_model):
    
    if model.name == "resnet50" or model.name == "mobilenet_1.00_224":
        # prepare the image for the VGG model
        processed_image = image.copy()

    elif model.name == "vgg16":
        
        # prepare the image for the VGG model
        processed_image = vgg16.preprocess_input(image.copy())
        #processed_image = image_batch.copy()

    # get the predicted probabilities for each class
    predictions = model.predict(image)
    # print predictions

    # convert the probabilities to class labels
    # We will get top 5 predictions which is the default
    label = decode_predictions(predictions)
    
    return label

In [ ]:
load_from_file  = True

In [ ]:
x,y = 224,224

In [ ]:
if load_from_file:
    filename    = 'media/img0.png'
    original    = load_img(filename, target_size=(x, y))
    numpy_image = img_to_array(original)
    end_image   = np.expand_dims(numpy_image, 0)

In [ ]:
canvas = np.ones((x,y,3))*255


"""

STUFF HERE

"""


end_image = np.expand_dims(canvas,0)

In [ ]:
# convert the probabilities to class labels
# We will get top 5 predictions which is the default
labels = predict(end_image, model)

In [ ]:
plt.imshow(np.uint8(end_image[0]))
labels